In [1]:
import numpy as np
import pandas as pd

UG_courses = ["BOA","BOB","BOC","BOE","BSC"]
Df = pd.read_csv("synthetic_Data_truncated_normal_approach2.csv")
print("size of dataframe:",Df.shape)


size of dataframe: (24017, 108)


In [2]:
Df.columns[:30]
Df["COURSE_CODE"] = Df["COURSE_CODE"].fillna('not_taking')
Df = Df[Df.columns].fillna('0')
Df = Df[Df["COURSE_CODE"] !='not_taking']
print("size of dataframe:",Df.shape)

size of dataframe: (5437, 108)


In [3]:
Df.isnull().sum().sum()
#check if there is missing value

0

In [4]:
X = Df[Df.columns.difference(['CODE_DESCR_mast','COURSE_CODE'])]
print("size of dataframe:",Df.shape)
y = Df[['CODE_DESCR_mast','COURSE_CODE']]
print("X has size:",X.shape, " and y has size:",y.shape)

size of dataframe: (5437, 108)
X has size: (5437, 106)  and y has size: (5437, 2)


In [5]:
## reorder
X = X[Df.columns.difference(['CODE_DESCR_mast','COURSE_CODE'])]
X.columns[:30]

Index(['AGE_BAND', 'ATAR_BAND', 'BOAsubject1', 'BOAsubject10', 'BOAsubject11',
       'BOAsubject12', 'BOAsubject13', 'BOAsubject14', 'BOAsubject15',
       'BOAsubject16', 'BOAsubject2', 'BOAsubject3', 'BOAsubject4',
       'BOAsubject5', 'BOAsubject6', 'BOAsubject7', 'BOAsubject8',
       'BOAsubject9', 'BOBsubject1', 'BOBsubject10', 'BOBsubject11',
       'BOBsubject12', 'BOBsubject13', 'BOBsubject14', 'BOBsubject15',
       'BOBsubject16', 'BOBsubject2', 'BOBsubject3', 'BOBsubject4',
       'BOBsubject5'],
      dtype='object')

In [6]:
subject_course_list = []
for course in UG_courses:
    for i in range(0,16):
        subject_course_list += [course+'subject'+str(i+1)]
profile_title_list = X.columns.difference(subject_course_list)
print(profile_title_list)

Index(['AGE_BAND', 'ATAR_BAND', 'CODE_DESCR_bach', 'COURSE_CREDITS_REQ_bach',
       'COURSE_CREDITS_REQ_mast', 'FACULTY_CODE_bach', 'FACULTY_CODE_mast',
       'HOME_LANGUAGE', 'IS_GENDER_MALE', 'IS_INTERNATIONAL',
       'LANGUAGE_TEST_SCORE', 'LANGUAGE_TEST_TYPE', 'LOCATION_CODE_bach',
       'LOCATION_CODE_mast', 'Mean', 'N_SUBJECTS_bach', 'N_SUBJECTS_mast',
       'STATE', 'STUDY_YEAR_bach', 'STUDY_YEAR_mast', 'STU_ID_PROXY',
       'UG_course', 'Unnamed: 0', 'Variance', 'index_order', 'top5_bach_bach'],
      dtype='object')


In [7]:
useless_profile_title_list = ['STU_ID_PROXY','UG_course','CODE_DESCR_bach','FACULTY_CODE_mast','LANGUAGE_TEST_SCORE','LANGUAGE_TEST_TYPE']+['FACULTY_CODE_mast']
# data cleaning, to sparse, or useless
useful_profile_title_list= list(set(profile_title_list)-set(useless_profile_title_list))
column_title_need_onehotEncode = ['COURSE_CREDITS_REQ_bach','FACULTY_CODE_bach','LOCATION_CODE_bach','top5_bach_bach',
                                  'AGE_BAND','STATE','HOME_LANGUAGE','ATAR_BAND',
                                  'COURSE_CREDITS_REQ_mast','LOCATION_CODE_mast','STUDY_YEAR_mast','N_SUBJECTS_mast']
X.isnull().sum().sum()



0

In [8]:

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

####binary encoder
encoder = LabelBinarizer()
Y_values = encoder.fit_transform(y["COURSE_CODE"].values)

X_part1 = pd.DataFrame({})
for column in column_title_need_onehotEncode:
    column_values=X[column_title_need_onehotEncode][column].astype(str)
    binarised_x = encoder.fit_transform(column_values.values)
    X_part1 = pd.concat([X_part1,pd.DataFrame(binarised_x)],axis=1)

##### one hot encoder   
    # enc = preprocessing.OneHotEncoder(categories='auto')  
# X_part1 = enc.fit_transform(X_part1.values).toarray()
    
X_part2 = X[list(set(useful_profile_title_list)-set(column_title_need_onehotEncode))]
#X_values = pd.concat([pd.DataFrame(X_part1),X_part2],axis = 1)
X_values = pd.DataFrame(np.hstack([pd.DataFrame(X_part1),X_part2]))
print(X_part1.shape)
print(X_part2.shape)
print(X_values.shape)
print(X[useful_profile_title_list].shape)


(5437, 80)
(5437, 8)
(5437, 88)
(5437, 20)


In [9]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, Y_train, Y_test = train_test_split(X_values.values, Y_values, test_size=0.33, random_state=27)


# modelling


In [10]:

import timeit

start = timeit.default_timer()
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

cls = DecisionTreeClassifier()
cls.fit(X_train, Y_train)
Y_pred = cls.predict(X_test)

print("weight f1 score:",metrics.f1_score(Y_test, Y_pred, average="weighted"))
print("sample f1 score:",metrics.f1_score(Y_test, Y_pred, average="samples"))



stop = timeit.default_timer()
print('Time in seconds: ', stop - start) 

weight f1 score: 0.5436880815728787
sample f1 score: 0.5442896935933148
Time in seconds:  0.8912377350000042


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [11]:
import timeit

start = timeit.default_timer()

from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
cls.fit(X_train, Y_train)
Y_pred = cls.predict(X_test)

print("weight f1 score:",metrics.f1_score(Y_test, Y_pred, average="weighted"))
print("sample f1 score:",metrics.f1_score(Y_test, Y_pred, average="samples"))



stop = timeit.default_timer()
print('Time in seconds: ', stop - start) 

weight f1 score: 0.5403695906067665
sample f1 score: 0.5426183844011142
Time in seconds:  0.701679545999923


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [12]:
import timeit

start = timeit.default_timer()

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
cls.fit(X_train, Y_train)
Y_pred = cls.predict(X_test)

print("weight f1 score:",metrics.f1_score(Y_test, Y_pred, average="weighted"))
print("sample f1 score:",metrics.f1_score(Y_test, Y_pred, average="samples"))



stop = timeit.default_timer()
print('Time in seconds: ', stop - start) 


weight f1 score: 0.54407178946186
sample f1 score: 0.5437325905292479
Time in seconds:  0.7021464980000474


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [13]:
print("type of master degree:",len(pd.DataFrame(Df["CODE_DESCR_mast"]).groupby(
    pd.DataFrame(Df["CODE_DESCR_mast"]).columns.tolist(),as_index=False)))
# pd.DataFrame(Df["CODE_DESCR_mast"]).groupby(pd.DataFrame(Df["CODE_DESCR_mast"]).columns.tolist(),as_index=False).size()


type of master degree: 166


In [14]:
print("type of master degree:",len(pd.DataFrame(Y_train).groupby(
    pd.DataFrame(Y_train).columns.tolist(),as_index=False)))

type of master degree: 170


In [15]:
import timeit

start = timeit.default_timer()

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=len(pd.DataFrame(Y_train).groupby(
    pd.DataFrame(Y_train).columns.tolist(),as_index=False)))
neigh.fit(X_train, Y_train)
Y_pred = neigh.predict(X_test)

print("weight f1 score:",metrics.f1_score(Y_test, Y_pred, average="weighted"))
print("sample f1 score:",metrics.f1_score(Y_test, Y_pred, average="samples"))



stop = timeit.default_timer()
print('Time in seconds: ', stop - start) 


weight f1 score: 0.03300945320553594
sample f1 score: 0.02395543175487465
Time in seconds:  9.390609622000056


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [16]:
import timeit
start = timeit.default_timer()

from sklearn.model_selection import GridSearchCV
grid_params = {'n_neighbors':[5,10,20,50,80,100,140],
              'weights':['uniform','distance'],
              'metric':['euclidean','manhattan']}
grid = GridSearchCV(KNeighborsClassifier(),grid_params,verbose = 1,cv=3,n_jobs = -1)
result  = grid.fit(X_train, Y_train)

stop = timeit.default_timer()
print('Time in seconds: ', stop - start) 

Fitting 3 folds for each of 28 candidates, totalling 84 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.3min


Time in seconds:  251.37297231000002


[Parallel(n_jobs=-1)]: Done  84 out of  84 | elapsed:  4.2min finished


In [17]:
best_parameters = grid.best_params_
print("best parameters are:",best_parameters)
best_result = grid.best_score_
print("best result is :",best_result)

best parameters are: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
best result is : 0.1331685886875343


In [18]:
import timeit
start = timeit.default_timer()

from sklearn.model_selection import GridSearchCV
grid_params = {'n_neighbors':[50,60,65,70,75,80,85,90,100],
              'weights':['uniform'],
              'metric':['euclidean','manhattan']}
grid = GridSearchCV(KNeighborsClassifier(),grid_params,verbose = 1,cv=3,n_jobs = -1)
result  = grid.fit(X_train, Y_train)

best_parameters = grid.best_params_
print("best parameters are:",best_parameters)
best_result = grid.best_score_
print("best result is :",best_result)

stop = timeit.default_timer()
print('Time in seconds: ', stop - start) 

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  4.3min


best parameters are: {'metric': 'euclidean', 'n_neighbors': 65, 'weights': 'uniform'}
best result is : 0.04777594728171334
Time in seconds:  303.025681933


[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:  5.0min finished


In [ ]:
result  = grid.fit(X_train, Y_train)


In [ ]:
grid_mean_scores = [result.mean_validation_score for result in grid.grid_scores_]
plt.plot(k_range, grid_mean_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')

# Gradient_Boosting_Classifier

In [19]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y_values = label_encoder.fit_transform(y["COURSE_CODE"].values)


X_part1 = pd.DataFrame({})
for column in column_title_need_onehotEncode:
    column_values=X[column_title_need_onehotEncode][column].astype(str)
    label_encoder_x = label_encoder.fit_transform(column_values.values)
    X_part1 = pd.concat([X_part1,pd.DataFrame(label_encoder_x)],axis=1)

##### one hot encoder   
    # enc = preprocessing.OneHotEncoder(categories='auto')  
# X_part1 = enc.fit_transform(X_part1.values).toarray()
    
X_part2 = X[list(set(useful_profile_title_list)-set(column_title_need_onehotEncode))]
#X_values = pd.concat([pd.DataFrame(X_part1),X_part2],axis = 1)
X_values = pd.DataFrame(np.hstack([pd.DataFrame(X_part1),X_part2]))
print(X_part1.shape)
print(X_part2.shape)
print(X_values.shape)
print(X[useful_profile_title_list].shape)
print(Y_values.shape)

(5437, 12)
(5437, 8)
(5437, 20)
(5437, 20)
(5437,)


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_values.values, Y_values, test_size=0.33, random_state=27)
print(Y_train.shape)

(3642,)


In [21]:
import timeit
from sklearn.ensemble import GradientBoostingClassifier
start = timeit.default_timer()
gbc = GradientBoostingClassifier(loss= 'deviance', learning_rate=0.01, n_estimators= 50, random_state=0)
gbc.fit(X_train, Y_train)
Y_pred = gbc.predict(X_test)

print("weight f1 score:",metrics.f1_score(Y_test, Y_pred, average="weighted"))
#print("sample f1 score:",metrics.f1_score(Y_test, Y_pred, average="samples"))



stop = timeit.default_timer()
print('Time in seconds: ', stop - start)

weight f1 score: 0.4548877727392068
Time in seconds:  148.40280601200038


In [22]:
import timeit
start = timeit.default_timer()
from sklearn.model_selection import GridSearchCV
grid_params = {'loss':['deviance', 'exponential'],
              'learning_rate':[0.01, 0.1, 1],
              'n_estimators':[10,40,80,100]}
gbc_grid = GridSearchCV(GradientBoostingClassifier(random_state=0),grid_params,verbose = 1,cv=3,n_jobs = -1)
result  = gbc_grid.fit(X_train, Y_train)

best_parameters = gbc_grid.best_params_
print("best parameters are:",best_parameters)
best_result = gbc_grid.best_score_
print("best result is :",best_result)

stop = timeit.default_timer()
print('Time in seconds: ', stop - start)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 31.5min
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed: 49.0min finished


best parameters are: {'learning_rate': 0.01, 'loss': 'deviance', 'n_estimators': 100}
best result is : 0.5823723228995057
Time in seconds:  3239.2904523429997


In [23]:
Y_pred = gbc_grid.predict(X_test)
print("weight f1 score:",metrics.f1_score(Y_test, Y_pred, average="weighted"))
#print("results: ", gbc_grid.cv_results_)

weight f1 score: 0.5332102284180733


#MultinomialNB

In [24]:
import timeit
from sklearn.naive_bayes import MultinomialNB
start = timeit.default_timer()
mnb = MultinomialNB()
mnb.fit(X_train, Y_train)
Y_pred = mnb.predict(X_test)

print("weight f1 score:",metrics.f1_score(Y_test, Y_pred, average="weighted"))
#print("sample f1 score:",metrics.f1_score(Y_test, Y_pred, average="samples"))



stop = timeit.default_timer()
print('Time in seconds: ', stop - start)

weight f1 score: 0.011768190362189056
Time in seconds:  0.03485899299994344


In [25]:
import timeit
start = timeit.default_timer()
from sklearn.model_selection import GridSearchCV
grid_params = {'alpha':[0.001,0.01,.1,1],
              'fit_prior':[True]}
mnb_grid = GridSearchCV(MultinomialNB(),grid_params,verbose = 1,cv=3,n_jobs = -1)
result  = mnb_grid.fit(X_train, Y_train)

best_parameters = mnb_grid.best_params_
print("best parameters are:",best_parameters)
best_result = mnb_grid.best_score_
print("best result is :",best_result)

stop = timeit.default_timer()
print('Time in seconds: ', stop - start)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


best parameters are: {'alpha': 0.001, 'fit_prior': True}
best result is : 0.018396485447556288
Time in seconds:  1.0285455829998682


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.0s finished


In [26]:
Y_pred = mnb_grid.predict(X_test)
print("weight f1 score:",metrics.f1_score(Y_test, Y_pred, average="weighted"))

weight f1 score: 0.013421739876255829
